In [1]:
from paddlenlp.transformers import AutoModelForSequenceClassification, AutoTokenizer
import paddle.nn.functional as F
import paddle
import re
model1=AutoModelForSequenceClassification.from_pretrained("work/model/",num_classes=6)
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

[2024-03-05 16:57:09,273] [    INFO] - We are using <class 'paddlenlp.transformers.bert.modeling.BertForSequenceClassification'> to load 'work/model/'.
[2024-03-05 16:57:20,647] [    INFO] - We are using <class 'paddlenlp.transformers.bert.tokenizer.BertTokenizer'> to load 'bert-base-chinese'.
[2024-03-05 16:57:20,651] [    INFO] - Downloading https://bj.bcebos.com/paddle-hapi/models/bert/bert-base-chinese-vocab.txt and saved to /home/aistudio/.paddlenlp/models/bert-base-chinese
[2024-03-05 16:57:20,654] [    INFO] - Downloading bert-base-chinese-vocab.txt from https://bj.bcebos.com/paddle-hapi/models/bert/bert-base-chinese-vocab.txt
100%|██████████| 107k/107k [00:00<00:00, 1.61MB/s]
[2024-03-05 16:57:21,083] [    INFO] - tokenizer config file saved in /home/aistudio/.paddlenlp/models/bert-base-chinese/tokenizer_config.json
[2024-03-05 16:57:21,087] [    INFO] - Special tokens file saved in /home/aistudio/.paddlenlp/models/bert-base-chinese/special_tokens_map.json


In [2]:
class TokenItem:
    def __init__(self,sentence,tokenizer):
        self.tokenizer=tokenizer
        self.token=tokenizer(
            [sentence],
            max_length=512,
            padding='max_length'
        )
    def __del__(self):
        pass

def get_sentiment(label):
    hashmap={0:"愤怒",1:"恐惧",2:"积极",3:"无情绪",4:"悲伤",5:"惊奇"}
    return hashmap[label]

def scentence_preprocessing(sentence):
    return re.sub(r'#.*?#', '', sentence)

def predict_sentence(sentence):
    with paddle.no_grad():
        tokenItem=TokenItem(sentence,tokenizer)
        outputs = model1(input_ids=paddle.to_tensor(tokenItem.token.input_ids,dtype='int64'),
                            token_type_ids=paddle.to_tensor(tokenItem.token.token_type_ids,dtype='int64'))
        del tokenItem
        result = paddle.argmax(outputs[0])
        label=result.numpy()
        sentiment=get_sentiment(int(label))
        out1 = F.softmax(outputs[0])
        
    return sentiment,out1


In [3]:
print(predict_sentence("🙋"))

('无情绪', Tensor(shape=[6], dtype=float32, place=Place(cpu), stop_gradient=True,
       [0.06043121, 0.01692171, 0.16828093, 0.33182356, 0.11378976, 0.30875286]))
